## Lesson-01

# 本节内容
- ## 课程概览
- ## 人工智能与NLP
- ## 基于规则的语言模型
- ## 基于概率的语言模型

# 一、课程概览

## 1.1 核心能力提升部分

In [1]:
simple_grammar = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*                  
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>  蓝色的 | 好看的 | 小小的
"""

## Adj* ==> null / Adj Adj  n Adj 

In [3]:
another_grammar = """
# 
"""

In [14]:
import random

In [5]:
def adj():  return random.choice('蓝色的 | 好看的 | 小小的'.split('|')).split()[0]
def adj_star():
    return random.choice([lambda : '', lambda : adj() + adj_star()])()

In [6]:
def adj_star():
    return random.choice([lambda : '', lambda : adj() + adj_star()])()

In [7]:
adj_star()

'好看的小小的'

## But the question is ? 

如果我们更换了语法，会发现所有写过的程序，都要重新写。:( 

In [9]:
adj_grammar = """
Adj* => null | Adj Adj*
Adj =>  蓝色的 | 好看的 | 小小的
"""

In [10]:
def create_grammar(grammar_str, split='=>', line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip(): continue
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar

In [11]:
grammar = create_grammar(adj_grammar)

In [12]:
grammar

{'Adj*': [['null'], ['Adj', 'Adj*']], 'Adj': [['蓝色的'], ['好看的'], ['小小的']]}

In [12]:
grammar['Adj*']

[['null'], ['Adj', 'Adj*']]

In [15]:
choice = random.choice

def generate(gram, target):
    if target not in gram: return target # means target is a terminal expression #1
    
    expaned = [generate(gram, t) for t in choice(gram[target])]  #2
    return ''.join([e if e != '/n' else '\n' for e in expaned if e != 'null']) #3

generate(gram,sentence) target=sentence 
expand=[]
=>1. if sentence not in gram  
=>2. gram[sentnece] = [['noun_phrase', 'verb_phrase']],  for t in [nount_phrase,verb_phrase]
    generate(gram,noun_phrase)
    ==>1. if noun_phrase not in gram   
    ==>2. gram[noun_phrase]=[['Article', 'Adj*', 'noun']] 随机选取一个（二维列表中只有一个元素，所以都要走一遍，达到遍历的效果） for t in ['Article', 'Adj*', 'noun']  
        generate(gram, Article)
        ==> 1. if Article not in gram
        ==> 2. gram[Article]= [['一个'], ['这个']] 随机选取一个（二选一）  for t in ["这个"]  t="这个"
          generate(gram,"这个")
          ==>1 if "这个" not  in gram  ==>  return "这个"
       genrate(gram,"Adj*")
       ==> 1. if Adj* not in gram
       ==> 2. gram["Adj*"] = [['null'], ['Adj', 'Adj*']]  t=["noun"]  t="noun"
          
expand =["一个","好看的" ,"小猫 "]
"一个好看的小猫"

In [16]:
example_grammar = create_grammar(simple_grammar)

In [17]:
example_grammar

{'sentence': [['noun_phrase', 'verb_phrase']],
 'noun_phrase': [['Article', 'Adj*', 'noun']],
 'Adj*': [['null'], ['Adj', 'Adj*']],
 'verb_phrase': [['verb', 'noun_phrase']],
 'Article': [['一个'], ['这个']],
 'noun': [['女人'], ['篮球'], ['桌子'], ['小猫']],
 'verb': [['看着'], ['坐在'], ['听着'], ['看见']],
 'Adj': [['蓝色的'], ['好看的'], ['小小的']]}

In [24]:
generate(gram=example_grammar, target='sentence')

'一个篮球看着一个女人'

In [25]:
#在西部世界里，一个”人类“的语言可以定义为：

human = """
human = 自己 寻找 活动
自己 = 我 | 俺 | 我们 
寻找 = 找找 | 想找点 
活动 = 乐子 | 玩的
"""


#一个“接待员”的语言可以定义为

host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = null
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？
"""

In [26]:
for i in range(20):
    print(generate(gram=create_grammar(host, split='='), target='host'))

小朋友,您好我是8号,您需要赌博吗？
小朋友,您好我是6号,请问你要喝酒吗？
小朋友,你好我是6号,请问你要打牌吗？
小朋友,您好我是5号,您需要赌博吗？
您好我是36号,请问你要打猎吗？
您好我是71712号,请问你要赌博吗？
你好我是78864号,请问你要喝酒吗？
女士,你好我是28号,您需要打猎吗？
先生,您好我是676号,您需要打猎吗？
小朋友,您好我是77号,请问你要赌博吗？
女士,您好我是2号,您需要喝酒吗？
小朋友,你好我是3号,您需要喝酒吗？
先生,你好我是5号,请问你要赌博吗？
您好我是91号,您需要赌博吗？
你好我是4号,请问你要喝酒吗？
小朋友,您好我是81952号,请问你要打牌吗？
你好我是918号,您需要赌博吗？
先生,你好我是23号,您需要喝酒吗？
你好我是7号,您需要打猎吗？
您好我是667号,您需要打牌吗？


In [ ]:
希望能够生成最合理的一句话？

## Data Driven

我们的目标是，希望能做一个程序，然后，当输入的数据变化的时候，我们的程序不用重写。Generalization.

AI? 如何能自动化解决问题，我们找到一个方法之后，输入变了，我们的这个方法，不用变。

In [1]:
simpel_programming = '''
programming => if_stmt | assign | while_loop
while_loop => while ( cond ) { change_line stmt change_line }
if_stmt => if ( cond )  { change_line stmt change_line } | if ( cond )  { change_line stmt change_line } else { change_line stmt change_line } 
change_line => /N
cond => var op var
op => | == | < | >= | <= 
stmt => assign | if_stmt
assign => var = var
var =>  var _ num | words 
words => words _ word | word 
word => name | info |  student | lib | database 
nums => nums num | num
num => 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 0
'''

In [2]:
print(generate(gram=create_grammar(simpel_programming, split='=>'), target='programming'))

NameError: name 'generate' is not defined

In [ ]:
def pretty_print(line):
    # utility tool function
    lines = line.split('/N')
    
    code_lines = []
    
    for i, sen in enumerate(lines):
        if i < len(lines) / 2: 
            #print()
            code_lines.append(i * "  " + sen)
        else:
            code_lines.append((len(lines) - i) * " " + sen)
    
    return code_lines

In [ ]:
generated_programming = []

for i in range(20):
    generated_programming += pretty_print(generate(gram=create_grammar(simpel_programming, split='=>'), target='programming'))

In [ ]:
for line in generated_programming:
    print(line)

# Language Model

$$ language\_model(String) = Probability(String) \in (0, 1) $$

$$ Pro(w_1 w_2 w_3 w_4) = Pr(w_1 | w_2 w_3 w_ 4) * P(w2 | w_3 w_4) * Pr(w_3 | w_4) * Pr(w_4)$$ 

$$ Pro(w_1 w_2 w_3 w_4) \sim Pr(w_1 | w_2 ) * P(w2 | w_3 ) * Pr(w_3 | w_4) * Pr(w_4)$$ 

how to get $ Pr(w1 | w2 w3 w4) $ ?

In [24]:
import random
import jieba

In [25]:
random.choice(range(100))

68

In [26]:
filename = '../../Data_source/sqlResult_1558435.csv'

In [27]:
import pandas as pd

In [28]:
content = pd.read_csv(filename, encoding='gb18030')

In [29]:
content.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [ ]:
articles = content['content'].tolist()

In [ ]:
len(articles)

词 ==》中文需要切词 =》 我想要玩游戏 =》 我  想要  玩  游戏 / i want to play games
jieba 
我  1
想要 1
玩 1
游戏 1

In [ ]:
import re

In [ ]:
def token(string):
    # we will learn the regular expression next course.
    return re.findall('\w+', string)

In [ ]:
from collections import Counter

In [ ]:
with_jieba_cut = Counter(jieba.cut(articles[110]))

In [ ]:
with_jieba_cut.most_common()[:10]

In [ ]:
''.join(token(articles[110]))

In [ ]:
articles_clean = [''.join(token(str(a)))for a in articles]

In [ ]:
len(articles_clean)

In [ ]:
with open('article_9k.txt', 'w') as f:
    for a in articles_clean:
        f.write(a + '\n')

In [ ]:
def cut(string): return list(jieba.cut(string))

In [ ]:
TOKEN = []

In [ ]:
for i, line in enumerate((open('article_9k.txt'))):
    if i % 100 == 0: print(i)
    
    # replace 10000 with a big number when you do your homework. 
    
    if i > 10000: break    
    TOKEN += cut(line)

In [ ]:
from functools import reduce

In [ ]:
from operator import add, mul

In [ ]:
reduce(add, [1, 2, 3, 4, 5, 8])

In [ ]:
[1, 2, 3] + [3, 43, 5]

In [ ]:
from collections import Counter

In [ ]:
words_count = Counter(TOKEN)

In [ ]:
words_count.most_common(100)

In [ ]:
frequiences = [f for w, f in words_count.most_common(100)]

In [ ]:
x = [i for i in range(100)]

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x, frequiences)

In [ ]:
import numpy as np

In [ ]:
plt.plot(x, np.log(frequiences))

In [ ]:
def prob_1(word):
    return words_count[word] / len(TOKEN)

# count(wk)/(number of words)

p(w1|w2) = count(w1,w2)/count（w1)

In [ ]:
prob_1('我们')

In [ ]:
TOKEN[:10]

In [ ]:
TOKEN = [str(t) for t in TOKEN]

In [ ]:
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]

In [ ]:
TOKEN_2_GRAM[:10]

In [ ]:
words_count_2 = Counter(TOKEN_2_GRAM)

In [ ]:
def prob_1(word): return words_count[word] / len(TOKEN)

In [ ]:
def prob_2(word1, word2):  # p(w1,w2) = count(w1,2)/count(w1)
    if word1 + word2 in words_count_2: return words_count_2[word1+word2] / words_count[word1]
    else:
        return 1 / len(TOKEN_2_GRAM)
    
#  (w1 w2), (w3,w4) (w4,w5)  2-gram
# (w1,w3)  1/3

In [ ]:
prob_2('我们', '在')

In [ ]:
prob_2('在', '吃饭')

In [ ]:
prob_2('去', '吃饭')

In [ ]:
def get_probablity(sentence):
    words = cut(sentence)
    
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):   # word当前词
        next_ = words[i+1]                  # next_下一个词
        
        probability = prob_2(word, next_)  # p(w1|w2)
        
        sentence_pro *= probability  # p(s) = p(w_1)p(w2|w1)*p(w3|w2)..p(wn|wn-1)  p(w_1)可去掉，实际情况在句子中w1常常是句号，不影响结果
    
    return sentence_pro

In [ ]:
get_probablity('小明今天抽奖抽到一台苹果手机')

In [ ]:
get_probablity('小明今天抽奖抽到一架波音飞机')

In [ ]:
get_probablity('洋葱奶昔来一杯')

In [ ]:
get_probablity('养乐多绿来一杯')

In [ ]:
for sen in [generate(gram=example_grammar, target='sentence') for i in range(10)]:
    print('sentence: {} with Prb: {}'.format(sen, get_probablity(sen)))

In [ ]:
need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "今晚我去吃火锅 今晚火锅去吃我",
    "洋葱奶昔来一杯 养乐多绿来一杯"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = get_probablity(s1), get_probablity(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} with probility {}'.format(s1, p1))
    print('-'*4 + ' {} with probility {}'.format(s2, p2))